In [3]:
pip install gym==0.10

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\dcnlab\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
import copy
from sre_constants import SUCCESS
import numpy as np
import torch
import random
from matplotlib import pylab as plt
import gym
#from gym import wrappers 
from IPython.display import clear_output
import math
import torch.nn.functional as F

"""L1 = 2
L2 = 150
Lv = 100
La = 100
Lv_output = 1
La_output = 3
model = torch.nnSequential(
    torch.nn.Linear(L1, L2),
    torch.nn.ReLU(),
    torch.nn.Linear(L2, Lv),
    torch.nn.ReLU(),
    torch.nn.Linear(L2, La),
    torch.nn.ReLU(),
    torch.nn.Linear(Lv,Lv_output),
    torch.nn.Linear(La,La_output),
    
)"""

class NoisyFactorizedLinear(torch.nn.Linear):
    """
    NoisyNet layer with factorized gaussian noise
    N.B. nn.Linear already initializes weight and bias to
    """
    def __init__(self, in_features, out_features, sigma_zero=0.4, bias=True):
        super(NoisyFactorizedLinear, self).__init__(in_features, out_features, bias=bias)
        sigma_init = sigma_zero / math.sqrt(in_features)
        self.sigma_weight = torch.nn.Parameter(torch.Tensor(out_features, in_features).fill_(sigma_init))
        self.register_buffer("epsilon_input", torch.zeros(1, in_features))
        self.register_buffer("epsilon_output", torch.zeros(out_features, 1))
        if bias:
            self.sigma_bias = torch.nn.Parameter(torch.Tensor(out_features).fill_(sigma_init))

    def forward(self, input):
        bias = self.bias
        func = lambda x: torch.sign(x) * torch.sqrt(torch.abs(x))

        with torch.no_grad():
            torch.randn(self.epsilon_input.size(), out=self.epsilon_input)
            torch.randn(self.epsilon_output.size(), out=self.epsilon_output)
            eps_in = func(self.epsilon_input)
            eps_out = func(self.epsilon_output)
            noise_v = torch.mul(eps_in, eps_out).detach()
        if bias is not None:
            bias = bias + self.sigma_bias * eps_out.t()
        """print("epsilon_input:",self.epsilon_input.size())
        print("epsilon_output:",self.epsilon_output.size())
        print("weight.size:",self.weight.size())
        print("sigma_weight:",self.sigma_weight.size())
        print("noise_v:",noise_v.size())
        print("input:",input.size())"""
        #print("weight*noise:",(self.sigma_weight*noise_v).size())
        print("bias:",bias.size())
        return F.linear(input, self.weight + self.sigma_weight*noise_v, bias.squeeze(0))
        #return F.linear(input, self.weight, bias)

class Noisy_DQN(torch.nn.Module):
    def __init__(self):
        """
        Initialize a deep Q-learning network as described in
        https://storage.googleapis.com/deepmind-data/assets/papers/DeepMindNature14236Paper.pdf
        Arguments:
            in_channels: number of channel of input.
                i.e The number of most recent frames stacked together as describe in the paper
            num_actions: number of action-value to output, one-to-one correspondence to action in game.
        """
        super(Noisy_DQN, self).__init__()
        
        self.fc1 = torch.nn.Linear(2, 150)
        #self.fc1 = NoisyFactorizedLinear(2, 150)
        #self.fc2 = torch.nn.Linear(150, 100)
        self.fc2 = NoisyFactorizedLinear(150, 100)
        self.fc3 = NoisyFactorizedLinear(100, 3)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        #print("x:",x.size())
        #print("x.view",x.view(x.size(0), -1).size())
        #x = self.relu(self.fc1(x.view(x.size(0), -1)))
        #x = self.relu(self.fc2(x.view(x.size(0), -1)))
        x=self.relu(self.fc1(x))
        x=self.relu(self.fc2(x))
        return self.fc3(x)
    

"""class model_t(torch.nn.Module):
    def __init__(self):                                                         
        super(model_t, self).__init__()                                             

        self.feature_layer = torch.nn.Sequential(
            torch.nn.Linear(2, 150),                                                
            torch.nn.ReLU(),          
            torch.nn.Linear(150, 100),                                                
            torch.nn.ReLU(), 
        )
        
        self.advantage_layer = torch.nn.Sequential(
            torch.nn.Linear(100, 3),
            #torch.nn.ReLU(),                                                                            
        )
        
        self.value_layer = torch.nn.Sequential(
            torch.nn.Linear(100, 1),
            #torch.nn.ReLU(),                                                                                
        )
                                   
    def forward(self, x):                                                       
        feature = self.feature_layer(x)           
        value = self.value_layer(feature)
        advantage = self.advantage_layer(feature)
                                            
        q = value + advantage - advantage.mean(dim=-1, keepdim=True)
        
        return q"""

model=Noisy_DQN()
model2 = copy.deepcopy(model) #target network
model2.load_state_dict(model.state_dict()) #將原始Q網路中的參數複製給目標網路
loss_fn = torch.nn.MSELoss()
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

gamma = 0.9
epsilon = 0.6

env = gym.envs.make('MountainCar-v0')
env = env.unwrapped
#env = wrappers.Monitor(env,"C:\testml\video",video_callable=lambda count:count%10==0)
#env.Monitor.start('C:\testml\video',video_callable=lambda count: count % 10 == 0)

from collections import deque
epochs = 800 #訓練次數
losses = []
step_num = []
qval_record=[]
mem_size = 1000 
batch_size = 64 #batch_size
replay = deque(maxlen=mem_size) #replay buffer
max_moves = 1000 #每場遊戲最多可以走幾步
sync_freq = 200
success_flag=0
success_num=0
j=0
for i in range(epochs):
    state1_ = env.reset()
    state1 = torch.from_numpy(state1_).float()
    status = 1
    success_flag=0
    mov = 0     #記錄移動的步數
    while(status == 1): 
        j += 1
        mov += 1
        qval = model(state1) #輸出各動作的Q值
        #print(qval,i)
        qval_ = qval.data.numpy()
        #qval_record.append(np.max(qval_))
        """if (random.random() < epsilon):
            action_ = np.random.randint(0,2)
        else:
            action_ = np.argmax(qval_)"""
        action_ = np.argmax(qval_)
        state2_, reward , done , info= env.step(action_)
        #env.render() 
        #print(state2_)
        """
        if reward != -1:
            #step_num.append(mov)
            success_flag=1
            success_num+=1
            reward=200
        """

        if mov == 700 :
            reward = reward -  0.5
        if state2_[0] >= -0.4 :
            reward +=  0.2 + state2_[0]
        if state2_[1] >= 0.02 :
            reward +=   0.2 + state2_[1]*10

        if state2_[0] > 0.5 and success_flag == 0:
            status = 0
            #reward += 20
            success_num+=1
            step_num.append(mov)
            success_flag=1
        
        #if state2_[0] > 0.5:
                #reward = 5
        #elif state2_[0] <= -0.6:
            #reward = -1 + state2_[0]
        state2 = torch.from_numpy(state2_).float()
        #print(i)
        exp = (state1, action_, reward, state2, done)
        replay.append(exp)
        state1 = state2 
        #print(state1)
        #print(len(replay),batch_size)     

        if len(replay) > batch_size:
            minibatch = random.sample(replay, batch_size)
            state1_batch_ = torch.cat([s1 for (s1,a,r,s2,d) in minibatch])
            action_batch = torch.Tensor([a for (s1,a,r,s2,d) in minibatch])
            reward_batch = torch.Tensor([r for (s1,a,r,s2,d) in minibatch])
            state2_batch_ = torch.cat([s2 for (s1,a,r,s2,d) in minibatch])
            done_batch = torch.Tensor([d for (s1,a,r,s2,d) in minibatch])
            state1_batch = torch.reshape(state1_batch_, (64, 2))
            state2_batch = torch.reshape(state2_batch_, (64, 2))

            Q1 = model(state1_batch) 
            with torch.no_grad():     
                Q2 = model2(state2_batch) 
            Y = reward_batch + gamma * (1-done_batch)*torch.max(Q2,dim=1)[0]
            X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step() 
            losses.append(loss.item())    

            if j % sync_freq == 0:
                model2.load_state_dict(model.state_dict())
            
        if reward == 200 or mov > max_moves:
            status = 0 
            mov = 0
        #losses.append(loss.item())  
    if epsilon > 0.001: 
        epsilon -= (1/epochs) 
    """if i % 100 == 0:
        if i!=0 and i%1000 == 0 :
            print(i,loss)
        else:
            print(i)"""
    if success_flag == 0:
        step_num.append(1000)
    if i%20 == 0:
      print(i, loss.item(),success_num)
    #print('\r')
    #print(i, loss.item(),success_num)
    #print('\r\b')
#env.close()
#test_model(model)
print(i, loss.item(),success_num)
plt.figure(figsize=(10,7))
plt.plot(losses)
plt.xlabel("Steps",fontsize=11)
plt.ylabel("Loss",fontsize=11)

plt.figure(figsize=(10,7))
plt.plot(step_num)
plt.xlabel("epochs",fontsize=11)
plt.ylabel("action number",fontsize=11)

"""plt.figure(figsize=(10,7))
plt.plot(qval_record)
plt.xlabel("Steps",fontsize=11)
plt.ylabel("qval",fontsize=11)"""




WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size([1, 100])
bias: torch.Size([1, 3])
bias: torch.Size(

KeyboardInterrupt: 

In [ ]:
"""def test_model(model):
  i = 0
  env = gym.envs.make('MountainCar-v0')
  env = env.unwrapped
  for i in range(10):
    state3_ = env.reset()  
    env.render()
    state3 = torch.from_numpy(state3_).float()
    status = 1
    mov = 0
    while(status == 1):    
      mov += 1
      qval = model(state3)
      qval_ = qval.data.numpy()
      action_ = np.argmax(qval_) 
      print(action_)
      observation, reward, done, info = env.step(action_)
      state3=torch.from_numpy(observation).float()
      #print(action_,reward,done)
      #print(observation)
      env.render()
      if mov >= 1000:
        status=0
      #time.sleep(0.02)
      #env.render()    
    print(i)
  env.close()"""

In [ ]:

#env = gym.envs.make('MountainCar-v0')
#env = env.unwrapped
test_s=0
for i in range(1000):
    state3_ = env.reset()  
    #env.render()
    state3 = torch.from_numpy(state3_).float()
    status = 1
    mov = 0 
    while(status == 1):    
        mov += 1
        qval = model(state3)
        qval_ = qval.data.numpy()
        """if (random.random() < 0.1):
            action_ = np.random.randint(0,2)
        else:
            action_ = np.argmax(qval_)"""
        action_ = np.argmax(qval_)
        #state2_, reward , done , info= env.step(action_)
        #action_ = np.argmax(qval_) 
        #print(action_)
        observation, reward, done, info = env.step(action_)
        state3=torch.from_numpy(observation).float()
        #print(action_,reward,done)
        #print(observation)
        #env.render()
        if observation[0] > 0.5 :
            status = 0
            test_s += 1
        if mov >= 1000:
            status=0
        
        #env.render()    
    #print(i)
print(test_s)
#env.close()

In [ ]:
#test_model(model)
torch.max(Q2,dim=1)

In [ ]:
for name,param in model.named_parameters():
    print(name,param.requires_grad)
    #print(param)
#print(model.fc2.weight,model.fc2.sigma_weight,model.fc2.noise_v)